# 生成电视剧剧本

在这个项目中，你将使用 RNN 创作你自己的[《辛普森一家》](https://zh.wikipedia.org/wiki/%E8%BE%9B%E6%99%AE%E6%A3%AE%E4%B8%80%E5%AE%B6)电视剧剧本。你将会用到《辛普森一家》第 27 季中部分剧本的[数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)。你创建的神经网络将为一个在 [Moe 酒馆](https://simpsonswiki.com/wiki/Moe's_Tavern)中的场景生成一集新的剧本。
## 获取数据
我们早已为你提供了数据。你将使用原始数据集的子集，它只包括 Moe 酒馆中的场景。数据中并不包括酒馆的其他版本，比如 “Moe 的山洞”、“燃烧的 Moe 酒馆”、“Moe 叔叔的家庭大餐”等等。

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## 探索数据
使用 `view_sentence_range` 来查看数据的不同部分。

In [18]:
view_sentence_range = (0, 10)
print(type(view_sentence_range))

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
#split() 通过指定分隔符对字符串进行切片，如果参数 num 有指定值，则分隔 num+1 个子字符串.这里以空格为分隔符对字符串进行切片。
print('Roughly the number of unique words （大约的单词总数）: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes （场景总数）: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene （平均每个场景包含的句子总数）: {}'.format(np.average(sentence_count_scene)))

#生成列表，包含每个句子。方式：嵌套的列表推导式，第一层遍历scenes中的每个scene，第二层遍历scene中的每个句子。
sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines 句子总数: {}'.format(len(sentences)))
#word_count_sentence数组保存了每个句子的长度
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line 每行句子的平均长度: {}'.format(np.average(word_count_sentence)))

print()
#注意：view_sentence_range是一个元祖，在开始已经定义。关键知识点：print中遍历元祖的数据。
print('The sentences {} to {}:'.format(*view_sentence_range))
#查看第0-10行句子。注意第9、10行为空格
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

<class 'tuple'>
Dataset Stats
Roughly the number of unique words （大约的单词总数）: 11492
Number of scenes （场景总数）: 262
Average number of sentences in each scene （平均每个场景包含的句子总数）: 15.248091603053435
Number of lines 句子总数: 4257
Average number of words in each line 每行句子的平均长度: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.

## 实现预处理函数
对数据集进行的第一个操作是预处理。请实现下面两个预处理函数：

- 查询表
- 标记符号的字符串

### 查询表
要创建词嵌入，你首先要将词语转换为 id。请在这个函数中创建两个字典：

- 将词语转换为 id 的字典，我们称它为 `vocab_to_int`
- 将 id 转换为词语的字典，我们称它为 `int_to_vocab`

请在下面的元组中返回这些字典
 `(vocab_to_int, int_to_vocab)`

In [19]:
import numpy as np
import problem_unittests as tests
from collections import Counter

#知识点：枚举生成带序号的词汇字典。利用set()去重。
#enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab_to_int = {w:i for i,w in enumerate(set(text))}
    int_to_vocab = {i:w for i,w in enumerate(set(text))}
    print(vocab_to_int,'\n',int_to_vocab)
    print(len(vocab_to_int),len(int_to_vocab))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{"moe's": 0, 'yeah': 1, 'moe_szyslak': 2, 'another': 3, 'meet': 4, 'last': 5, 'days': 6, 'these': 7, 'rotch': 8, 'catch': 9, 'problems': 10, 'back': 11, 'eh': 12, "i'm": 13, 'barney_gumble': 14, 'of': 15, 'hello': 16, 'puke': 17, 'is': 18, 'your': 19, 'you': 20, 'with': 21, 'an': 22, 'only': 23, 'ice': 24, "you're": 25, 'enhance': 26, 'social': 27, 'matter': 28, 'carve': 29, 'tavern': 30, 'to': 31, 'hey': 32, 'one': 33, 'i': 34, 'give': 35, 'my': 36, 'listen': 37, 'elite': 38, 'normal': 39, 'the': 40, 'lately': 41, 'drink': 42, 'name': 43, 'pick': 44, 'homer': 45, 'whats': 46, 'got': 47, 'not': 48, 'has': 49, 'hold': 50, 'where': 51, 'should': 52, 'moe': 53, 'forget': 54, "i'll": 55, 'little': 56, 'on': 57, 'effervescent': 58, 'check': 59, 'self': 60, 'me': 61, 'there': 62, 'and': 63, 'seen': 64, 'gonna': 65, 'skills': 66, 'mike': 67, 'homer_simpson': 68, 'bart_simpson': 69, 'anybody': 70} 
 {0: "moe's", 1: 'yeah', 2: 'moe_szyslak', 3: 'another', 4: 'meet', 5: 'last', 6: 'days', 7: 'th

### 标记符号的字符串
我们会使用空格当作分隔符，来将剧本分割为词语数组。然而，句号和感叹号等符号使得神经网络难以分辨“再见”和“再见！”之间的区别。

实现函数 `token_lookup` 来返回一个字典，这个字典用于将 “!” 等符号标记为 “||Exclamation_Mark||” 形式。为下列符号创建一个字典，其中符号为标志，值为标记。

- period ( . )
- comma ( , )
- quotation mark ( " )
- semicolon ( ; )
- exclamation mark ( ! )
- question mark ( ? )
- left parenthesis ( ( )
- right parenthesis ( ) )
- dash ( -- )
- return ( \n )

这个字典将用于标记符号并在其周围添加分隔符（空格）。这能将符号视作单独词汇分割开来，并使神经网络更轻松地预测下一个词汇。请确保你并没有使用容易与词汇混淆的标记。与其使用 “dash” 这样的标记，试试使用“||dash||”。

In [20]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    #创建标点符号字典,“标点符号：\\英文名称\\”
    return  {
        '.': '||Period||',
        ',': '||Comma||',
        '"': '||Quotation_mark||',
        ';': '||Semicolon||',
        '!': '||Exclamation_mark||',
        '?': '||Question_mark||',
        '(': '||Left_Parentheses||',
        ')': '||Right_Parenthese||',
        '--': '||Dash||',
        '\n': '||Return||'
    }

print(token_lookup())
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

{'.': '||Period||', ',': '||Comma||', '"': '||Quotation_mark||', ';': '||Semicolon||', '!': '||Exclamation_mark||', '?': '||Question_mark||', '(': '||Left_Parentheses||', ')': '||Right_Parenthese||', '--': '||Dash||', '\n': '||Return||'}
Tests Passed


## 预处理并保存所有数据
运行以下代码将预处理所有数据，并将它们保存至文件。

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{'babe': 0, 'caught': 1, 'mis-statement': 2, 'mall': 3, 'alibi': 4, 'edna': 5, 'homer_doubles:': 6, 'prep': 7, 'rest': 8, 'supreme': 9, 'officer': 10, 'unsanitary': 11, 'absolut': 12, 'came': 13, "somethin'": 14, 'multiple': 15, 'fake': 16, 'wagering': 17, 'drink': 18, 'thank': 19, 'exultant': 20, 'giving': 21, 'burt': 22, 'true': 23, 'scornful': 24, 'referee': 25, 'majesty': 26, 'enabling': 27, 'supply': 28, 'cozies': 29, 'ivy-covered': 30, 'dame': 31, 'padre': 32, 'l': 33, 'spiritual': 34, 'icelandic': 35, 'slip': 36, 'man_with_tree_hat:': 37, 'fit': 38, 'regretted': 39, 'democracy': 40, 'jerk-ass': 41, 'orders': 42, "squeezin'": 43, 'determined': 44, 'log': 45, 'roach': 46, 'farewell': 47, 'fire': 48, 'saying': 49, 'priority': 50, 'sangre': 51, "father's": 52, 'brine': 53, 'story': 54, 'fruit': 55, 'non-american': 56, 'sodas': 57, 'd': 58, 'sister-in-law': 59, 'complicated': 60, 'lessons': 61, 'prints': 62, 'cool': 63, 'fudd': 64, 'lard': 65, 'huggenkiss': 66, 'playing': 67, 'badly'

# 检查点
这是你遇到的第一个检点。如果你想要回到这个 notebook，或需要重新打开 notebook，你都可以从这里开始。预处理的数据都已经保存完毕。

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## 创建神经网络
你将通过实现下面的函数，来创造用于构建 RNN 的必要元素：

- get_inputs
- get_init\_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### 检查 TensorFlow 版本并访问 GPU

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### 输入

实现函数 `get_inputs()` 来为神经网络创建 TF 占位符。它将创建下列占位符：

- 使用 [TF 占位符](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` 参量输入 "input" 文本占位符。
- Targets 占位符
- Learning Rate 占位符

返回下列元组中的占位符 `(Input, Targets, LearningRate)`

In [24]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32,shape=[None,None],name='input')
    targets = tf.placeholder(tf.int32,shape=[None,None],name='targets')
    learning_rate = tf.placeholder(tf.float32,name='learning_rate')
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### 创建 RNN Cell 并初始化

在 [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell) 中堆叠一个或多个 [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell)

- 使用 `rnn_size` 设定 RNN 大小。
- 使用 MultiRNNCell 的 [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) 函数初始化 Cell 状态
- 使用 [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity) 为初始状态应用名称 "initial_state"
 

返回 cell 和下列元组中的初始状态 `(Cell, InitialState)`

In [25]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #设置神经元被选中的概率keep_prob
    keep_prob = 0.8
    layers = 3
    #基本的LSTM循环网络单元
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #tf.nn.rnn_cell.DropoutWrapper()函数。cell：输入的循环神经网络的cell。
    #input_keep_prob：输入数据dropout的比率，如果是1，就不会执行dropout；
    #output_keep_prob：输出数据dropout的比率，如果是1，就不会执行dropout。
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    #对RNNCell进行堆叠。用tf.nn.rnn_cell MultiRNNCell创建3层RNN
    cell = tf.contrib.rnn.MultiRNNCell([drop] * layers)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    #简单来说tf.identity()就是返回一个一模一样的新的tensor。
    #Graph上不论是tensor还是operation的更新都要借助op来进行，而将一个tensor转化为op最简单的方法就是tf.identity()
    #参考解释：1）https://www.jianshu.com/p/6304b7c7896b 2）https://www.jianshu.com/p/9de22c907795
    initial_state = tf.identity(initial_state, name='initial_state')
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### 词嵌入
使用 TensorFlow 将嵌入运用到 `input_data` 中。
返回嵌入序列。

In [26]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    #根据均匀分布生成随机的vocab_size*embed_dim大小的矩阵。生成的值在[-1,1]范围内分布
    embeddings = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embeddings, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### 创建 RNN
你已经在 `get_init_cell()` 函数中创建了 RNN Cell。是时候使用这个 Cell 来创建 RNN了。

- 使用 [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) 创建 RNN
- 使用 [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity) 将名称 "final_state" 应用到最终状态中


返回下列元组中的输出和最终状态`(Outputs, FinalState)`

In [27]:
#定义RNN搭建函数
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs,final_state = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    #tf.identity函数返回一个向量。tf.identity(input,name=None)。name: A name for the operation (optional).
    final_state = tf.identity(final_state,'final_state')
    return outputs, final_state

    #operation是一个计算图的节点，用来运行在 tensor 上执行的计算操作，它接受零或多个 tensor 作为输入，
    #产生零或多个 Tensor 作为输出，简单的使用任意一个关联到 OP 创建器的操作都会成功的创建 OP 的对象实例，
    #比如说使用：c = tf.matmul(a, b)，就创建了一个类型为MatMul的 OP，它的输入是 a,b，输出是 c
    #参考资料：https://zhuanlan.zhihu.com/p/32399032

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### 构建神经网络
应用你在上面实现的函数，来：

- 使用你的 `get_embed(input_data, vocab_size, embed_dim)` 函数将嵌入应用到 `input_data` 中
- 使用 `cell` 和你的 `build_rnn(cell, inputs)` 函数来创建 RNN
- 应用一个完全联通线性激活和 `vocab_size` 的分层作为输出数量。

返回下列元组中的 logit 和最终状态 `Logits, FinalState`

In [28]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    #获取嵌入序列
    #embed = get_embed(input_data, vocab_size, rnn_size)
    embed = get_embed(input_data, vocab_size, rnn_size)
    #问题：outputs的格式？
    outputs, final_state = build_rnn(cell, embed)
    #Adds a fully connected layer，输入为outputs，输出单元数量为vocab_size。注意这里没采用激活函数以保持线性激活。
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### 批次

实现 `get_batches` 来使用 `int_text` 创建输入与目标批次。这些批次应为 Numpy 数组，并具有形状 `(number of batches, 2, batch size, sequence length)`。每个批次包含两个元素：

- 第一个元素为**输入**的单独批次，并具有形状 `[batch size, sequence length]`
- 第二个元素为**目标**的单独批次，并具有形状 `[batch size, sequence length]`

如果你无法在最后一个批次中填入足够数据，请放弃这个批次。

例如 `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` 将返回下面这个 Numpy 数组：

```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [29]:
#创建输入与目标批次
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    #计算批次数量，向下取整
    n_batches = len(int_text)//(batch_size*seq_length)
    
    #定义
    inputs = np.array(int_text[:n_batches*batch_size*seq_length])
    targets = np.array(int_text[1:n_batches*batch_size*seq_length+1])
    print('inputs1:.{}'.format(inputs))
    print('targets1:.{}'.format(targets))
    #concatenate：数组拼接函数。去掉targets中最后一个元素，拼接上inputs数组的第一个元素
    targets = np.concatenate([targets[:-1],[inputs[0]]])
    print('targets2:.{}'.format(targets))
    
    #先将inputs变成batch_size行，列数根据行数来定（-1）.
    #再用batch_size大小进行切分，纵向切分。split(ary, indices_or_sections, axis=0) 。
    x_batches = np.split(inputs.reshape(batch_size,-1),n_batches,1)
    
    print('reshape_input:.{}'.format(inputs.reshape(batch_size,-1)))
    print('x_batches_length x_batches中每个元素的大小:.{}'.format(len(x_batches)))
    print('x_batches[0][0]:.{}'.format(x_batches[0][0]))
    
    y_batches = np.split(targets.reshape(batch_size,-1),n_batches,1)
    
    #zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表。
    print('np.array:.{}'.format(np.array(list(zip(x_batches,y_batches))[0][0][0:4])))
          
    return np.array(list(zip(x_batches,y_batches)))
   
    #return np.array(result)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

inputs1:.[   0    1    2 ..., 4477 4478 4479]
targets1:.[   1    2    3 ..., 4478 4479 4480]
targets2:.[   1    2    3 ..., 4478 4479    0]
reshape_input:.[[   0    1    2 ...,   32   33   34]
 [  35   36   37 ...,   67   68   69]
 [  70   71   72 ...,  102  103  104]
 ..., 
 [4375 4376 4377 ..., 4407 4408 4409]
 [4410 4411 4412 ..., 4442 4443 4444]
 [4445 4446 4447 ..., 4477 4478 4479]]
x_batches_length x_batches中每个元素的大小:.7
x_batches[0][0]:.[0 1 2 3 4]
np.array:.[[  0   1   2   3   4]
 [ 35  36  37  38  39]
 [ 70  71  72  73  74]
 [105 106 107 108 109]]
Tests Passed


## 神经网络训练
### 超参数
调整下列参数:

- 将 `num_epochs` 设置为训练次数。
- 将 `batch_size` 设置为程序组大小。
- 将 `rnn_size` 设置为 RNN 大小。
- 将 `embed_dim` 设置为嵌入大小。
- 将 `seq_length` 设置为序列长度。
- 将 `learning_rate` 设置为学习率。
- 将 `show_every_n_batches` 设置为神经网络应输出的程序组数量。

In [30]:
# Number of Epochs
num_epochs = 300
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 150

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### 创建图表
使用你实现的神经网络创建图表。

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## 训练
在预处理数据中训练神经网络。如果你遇到困难，请查看这个[表格](https://discussions.udacity.com/)，看看是否有人遇到了和你一样的问题。

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

inputs1:.[5814  837 6554 ..., 4704  978 1227]
targets1:.[ 837 6554 2017 ...,  978 1227 5016]
targets2:.[ 837 6554 2017 ...,  978 1227 5814]
reshape_input:.[[5814  837 6554 ..., 3512 5494 5494]
 [5494 5814 3685 ..., 5494 4704 5364]
 [ 812  618 5487 ..., 5463 2348 4290]
 ..., 
 [5621 2792 1296 ..., 5727 4737  270]
 [5122 5494 5814 ..., 4103 1227 3346]
 [4201 1484 1227 ..., 4704  978 1227]]
x_batches_length x_batches中每个元素的大小:.22
x_batches[0][0]:.[5814  837 6554 2017 4879 2956 1505 3512 4583 5013  400 6381]
np.array:.[[5814  837 6554 2017 4879 2956 1505 3512 4583 5013  400 6381]
 [5494 5814 3685 4002 6653 5621 5013 6259 3512 5494 4704 4577]
 [ 812  618 5487 1227 6067 4394 5122 5494 4704 3604 4360 5494]
 [6634  885 5508 3512 5494 2253 3532  704  394  747 5723 3512]]
Epoch   0 Batch    0/22   train_loss = 8.822
Epoch   6 Batch   18/22   train_loss = 5.711
Epoch  13 Batch   14/22   train_loss = 5.071
Epoch  20 Batch   10/22   train_loss = 4.539
Epoch  27 Batch    6/22   train_loss = 4.145
Epo

## 储存参数
储存 `seq_length` 和 `save_dir` 来生成新的电视剧剧本。

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# 检查点

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## 实现生成函数
### 获取 Tensors
使用 [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name)函数从 `loaded_graph` 中获取 tensor。  使用下面的名称获取 tensor：

- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

返回下列元组中的 tensor `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)`

In [35]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    Initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    Final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, Initial_state, Final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### 选择词汇
实现 `pick_word()` 函数来使用 `probabilities` 选择下一个词汇。

In [36]:
#返回预测的单词
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #argmax函数返回的是最大数的索引
    print(type(probabilities))
    print(probabilities)
    index = np.argmax(probabilities)
    next_word = int_to_vocab[index]
    return next_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

<class 'numpy.ndarray'>
[ 0.1   0.8   0.05  0.05]
Tests Passed


## 生成电视剧剧本
这将为你生成一个电视剧剧本。通过设置 `gen_length` 来调整你想生成的剧本长度。

In [37]:
gen_length = 120
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
<class 'numpy.ndarray'>
[  2.45414551e-08   8.91510962e-11   6.16402207e-10 ...,   7.41243178e-10
   9.52552481e-10   9.72956715e-10]
<class 'numpy.ndarray'>
[  1.05341173e-10   9.06284353e-11   5.80767860e-08 ...,   2.76230483e-10
   1.06266356e-08   1.61317543e-10]
<class 'numpy.ndarray'>
[  5.10385678e-10   4.36026715e-10   1.49637344e-07 ...,   2.75274636e-10
   2.08302084e-10   1.62953984e-10]
<class 'numpy.ndarray'>
[  4.06697516e-14   4.89652704e-22   2.35265095e-16 ...,   1.93493623e-15
   2.60718033e-15   1.18239132e-13]
<class 'numpy.ndarray'>
[  2.45000166e-11   1.03978501e-11   2.37379374e-15 ...,   3.00934211e-08
   1.16890682e-07   6.83063883e-11]
<class 'numpy.ndarray'>
[  2.94242792e-15   3.33005699e-19   4.39650797e-18 ...,   1.00179163e-13
   1.89908527e-12   1.00760449e-08]
<class 'numpy.ndarray'>
[  4.35008514e-16   2.28073788e-12   1.28102037e-17 ...,   6.51409551e-11
   1.10836611e-10   1.65001013e-09]
<class 'numpy

# 这个电视剧剧本是无意义的
如果这个电视剧剧本毫无意义，那也没有关系。我们的训练文本不到一兆字节。为了获得更好的结果，你需要使用更小的词汇范围或是更多数据。幸运的是，我们的确拥有更多数据！在本项目开始之初我们也曾提过，这是[另一个数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)的子集。我们并没有让你基于所有数据进行训练，因为这将耗费大量时间。然而，你可以随意使用这些数据训练你的神经网络。当然，是在完成本项目之后。
# 提交项目
在提交项目时，请确保你在保存 notebook 前运行了所有的单元格代码。请将 notebook 文件保存为 "dlnd_tv_script_generation.ipynb"，并将它作为 HTML 文件保存在 "File" -> "Download as" 中。请将 "helper.py" 和 "problem_unittests.py" 文件一并提交。